In [1]:
import requests
import time
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords
from bs4 import BeautifulSoup 
import regex as re
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix,classification_report

In [2]:
new_df = pd.read_csv("cleaned_data.csv")

In [3]:
X= new_df['selftext']
y= new_df['subreddit']

In [4]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: subreddit, dtype: int64

In [5]:
X.head()

0    How come most Star Wars fans completly turn ar...
1    The latest 2 episodes of the clone wars are so...
2    So, to start it off i gotta tell that not just...
3    In the bar on Kijimi there is a character call...
4    At the beginning of the duel between Ahsoka an...
Name: selftext, dtype: object

In [6]:
#EDA

In [7]:
# Split the data into the training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.33,
                                                    stratify=y,
                                                    random_state=42)

In [8]:
y.value_counts(normalize = True)

0    0.515179
1    0.484821
Name: subreddit, dtype: float64

In [9]:
#Cleaning

In [10]:
# Import stopwords.
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/upasanamahanta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
print(CountVectorizer(stop_words='english').get_stop_words())

frozenset({'done', 'most', 'i', 'every', 'alone', 'thick', 'thru', 'side', 'other', 'if', 'former', 'two', 'until', 'onto', 'only', 'someone', 'should', 'yourselves', 'around', 'whatever', 'down', 'back', 'seeming', 'itself', 'out', 'full', 'wherein', 'also', 'will', 'do', 'beyond', 'have', 'him', 'neither', 'whole', 'here', 'might', 'she', 'yet', 'below', 'meanwhile', 'its', 'three', 'with', 'next', 'too', 'what', 'none', 'become', 'one', 'throughout', 'call', 'otherwise', 'rather', 'though', 'somehow', 'then', 'we', 'formerly', 'as', 'amongst', 'more', 'their', 'together', 'toward', 'hereupon', 'myself', 'anyone', 'indeed', 'beside', 'towards', 'me', 'sixty', 're', 'name', 'latterly', 'whether', 'empty', 'whereupon', 'however', 'twenty', 'us', 'something', 'whereafter', 'either', 'about', 'and', 'ltd', 'on', 'first', 'less', 'many', 'others', 'elsewhere', 'cant', 'besides', 'no', 'always', 'inc', 'front', 'anywhere', 'top', 'there', 'under', 'de', 'thus', 'much', 'themselves', 'among

In [12]:
# Let's look at sklearn's stopwords.
print(CountVectorizer(stop_words = 'english').get_stop_words())
print(stopwords.words('english'))

frozenset({'done', 'most', 'i', 'every', 'alone', 'thick', 'thru', 'side', 'other', 'if', 'former', 'two', 'until', 'onto', 'only', 'someone', 'should', 'yourselves', 'around', 'whatever', 'down', 'back', 'seeming', 'itself', 'out', 'full', 'wherein', 'also', 'will', 'do', 'beyond', 'have', 'him', 'neither', 'whole', 'here', 'might', 'she', 'yet', 'below', 'meanwhile', 'its', 'three', 'with', 'next', 'too', 'what', 'none', 'become', 'one', 'throughout', 'call', 'otherwise', 'rather', 'though', 'somehow', 'then', 'we', 'formerly', 'as', 'amongst', 'more', 'their', 'together', 'toward', 'hereupon', 'myself', 'anyone', 'indeed', 'beside', 'towards', 'me', 'sixty', 're', 'name', 'latterly', 'whether', 'empty', 'whereupon', 'however', 'twenty', 'us', 'something', 'whereafter', 'either', 'about', 'and', 'ltd', 'on', 'first', 'less', 'many', 'others', 'elsewhere', 'cant', 'besides', 'no', 'always', 'inc', 'front', 'anywhere', 'top', 'there', 'under', 'de', 'thus', 'much', 'themselves', 'among

In [13]:
y_train.head()

670     1
1033    1
67      0
903     1
1082    1
Name: subreddit, dtype: int64

In [14]:
X_train.head()

670     I'm guessing it is... and I had to read it a n...
1033    Hello !\n\nI am a huge fan of Star trek since ...
67       I think we can all agree that neither timelin...
903     And one question was, “why were the synths so ...
1082    First, before people think I'm going to basele...
Name: selftext, dtype: object

In [15]:
#EDA

In [16]:
def selftext_to_words(raw_selftext):
      
    # 1. Remove HTML.
    selftext_text = BeautifulSoup(raw_selftext).get_text()
    
    # 2. Remove non-letters.
    letters_only = re.sub("[^a-zA-Z]", " ", selftext_text)
    
    # 3. Convert to lower case, split into individual words.
    words = letters_only.lower().split()
    
    # a list, so convert the stopwords to a set.
    stops = set(stopwords.words('english'))
    
    # 5. Remove stopwords.
    meaningful_words = [w for w in words if w not in stops]
    
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return(" ".join(meaningful_words))


In [17]:
#Get the number of selftext based on the dataframe size.
total_selftext = new_df.shape[0]
print(f'There are {total_selftext} selftext.')

There are 1087 selftext.


In [18]:
 #Initialize an empty list to hold the clean selftext.
clean_train_selftexts = []
clean_test_selftexts = []

print("Cleaning and parsing the training set post selftext...")

# Instantiate counter.
j = 0

# For every review in our training set...
for train_selftext in X_train:
    
    # Convert text to words, then append to clean_train_selftext.
    clean_train_selftexts.append(selftext_to_words(train_selftext))
    
    # If the index is divisible by 1000, print a message.
    if (j + 1) % 100 == 0:
        print(f'selftext {j + 1} of {total_selftext}.')
    
    j += 1

# Let's do the same for our testing set.
print("Cleaning and parsing the testing set post selftext...")

# Instantiate counter.
j = 0

# For every review in our training set...
for test_selftext in X_test:
    
    # Convert text to words, then append to clean_train_selftext.
    clean_test_selftexts.append(selftext_to_words(test_selftext))
    
    # If the index is divisible by 1000, print a message.
    if (j + 1) % 100 == 0:
        print(f'selftext {j + 1} of {total_selftext}.')
    
    j += 1


Cleaning and parsing the training set post selftext...
selftext 100 of 1087.
selftext 200 of 1087.
selftext 300 of 1087.
selftext 400 of 1087.
selftext 500 of 1087.
selftext 600 of 1087.
selftext 700 of 1087.
Cleaning and parsing the testing set post selftext...
selftext 100 of 1087.
selftext 200 of 1087.
selftext 300 of 1087.


In [19]:
# Instantiate the "CountVectorizer" object, which is sklearn's
# bag of words tool.
vectorizer = CountVectorizer(analyzer = "word",
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = None,
                             max_features = 5000)

In [20]:

train_data_features = vectorizer.fit_transform(clean_train_selftexts)
test_data_features = vectorizer.transform(clean_test_selftexts)

In [21]:
print(train_data_features.shape)

(728, 5000)


In [22]:
print(test_data_features.shape)

(359, 5000)


In [23]:
vocab = vectorizer.get_feature_names()
print(vocab)

['aayla', 'abandoned', 'abandoning', 'abducted', 'abilities', 'ability', 'able', 'aboard', 'abrams', 'abruptly', 'absolute', 'absolutely', 'absurd', 'abusive', 'academy', 'accent', 'accept', 'acceptable', 'acceptance', 'accepted', 'accepting', 'access', 'accessible', 'accident', 'accidentally', 'accomplish', 'accomplished', 'according', 'achieve', 'across', 'act', 'acted', 'acting', 'action', 'actions', 'activate', 'active', 'actively', 'actor', 'actors', 'actress', 'acts', 'actual', 'actually', 'adam', 'adapt', 'add', 'added', 'adding', 'addition', 'additional', 'adds', 'adi', 'admiral', 'admit', 'admittedly', 'admonition', 'adopted', 'adoptive', 'ads', 'adult', 'adults', 'advance', 'advanced', 'advancements', 'advances', 'adventure', 'adventures', 'advice', 'aesthetic', 'af', 'affect', 'affects', 'afford', 'afraid', 'aftermath', 'afterwards', 'age', 'aged', 'agen', 'aggressive', 'agnes', 'ago', 'agree', 'agreed', 'agrees', 'ahead', 'ahsoka', 'ai', 'aim', 'aiming', 'air', 'aired', 'ai

In [24]:
#Logistic Regression

In [25]:
# Instantiate logistic regression model.
lr = LogisticRegression(solver = 'liblinear')#lr = LogisticRegression(solver = 'liblinear') # lbfgs vs. liblinear

# Fit model to training data.
lr.fit(train_data_features,y_train)

# Evaluate model on training data.
lr.score(train_data_features,y_train)

1.0

In [26]:
# Evaluate model on testing data.
lr.score(test_data_features,y_test)


0.9805013927576601

In [27]:
#Since our model has a higher accuracy on the training set than on the testing set, our model is overfit.
#We could combat this overfitting by removing features (i.e. setting max_features to be 5000) 
#and/or by regularizing our model

In [28]:
y_test.value_counts(normalize=True)

0    0.51532
1    0.48468
Name: subreddit, dtype: float64

In [29]:
# Let's set a pipeline up with two stages:
# 1. CountVectorizer (transformer)
# 2. LogisticRegression (estimator)

pipe = Pipeline([
    ('cvec', CountVectorizer()),
    ('lr', LogisticRegression(solver = 'lbfgs'))
])

In [30]:
# Search over the following values of hyperparameters:
# Maximum number of features fit: 2000, 3000, 4000, 5000
# Minimum number of documents needed to include token: 2, 3
# Maximum number of documents needed to include token: 90%, 95%
# Check (individual tokens) and also check (individual tokens and 2-grams).

pipe_params = {
    'cvec__max_features': [1_000,2_000,3_000,5_000],
    'cvec__min_df': [2, 3],
    'cvec__max_df': [.9, .95],
    'cvec__ngram_range': [(1,1), (1,2)]
}

In [31]:
# Instantiate GridSearchCV.

gs = GridSearchCV(pipe, 
                  param_grid=pipe_params,
                  cv=5) # 5-fold cross-validation.

In [32]:
# Fit GridSearch to training data.
gs.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('cvec',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        prep

In [33]:
# What's the best score?
print(gs.best_score_)

0.9628908833254606


In [34]:
# Save best model as gs_model.

gs_model = gs.best_estimator_

In [35]:
# Score model on training set.
gs_model.score(X_train, y_train)

1.0

In [36]:
# Score model on testing set.
gs_model.score(X_test, y_test)

0.9721448467966574

In [37]:
# Instantiate the transformer.
tvec = TfidfVectorizer()

In [38]:
df = pd.DataFrame(tvec.fit_transform(X_train).toarray(),
                  columns=tvec.get_feature_names())
df.head()

,000,007,00s,09,10,100,1000,10000,109,11,...,zhat,zilo,ziro,ziyal,zombie,zone,zoom,zorii,zv,zya
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.286711,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
X_train = tvec.fit_transform(X_train)

X_test = tvec.transform(X_test)

In [40]:
# Step 1: Instantiate our model.
logreg= LogisticRegression()

# Step 2: Fit our model.
logreg.fit(X_train,y_train)

print(f'Logistic Regression Intercept: {logreg.intercept_}')
print(f'Logistic Regression Coefficient: {logreg.coef_}')

Logistic Regression Intercept: [-0.20019556]
Logistic Regression Coefficient: [[ 0.03213119  0.00974824  0.0341927  ... -0.04091978  0.11873398
   0.0567927 ]]


In [41]:
# Instantiate logistic regression.
lr = LogisticRegression(solver = 'lbfgs')

# Fit logistic regression.
lr.fit(X_train, y_train)

# Evaluate logistic regression.
print(f'Training Score: {lr.score(X_train, y_train)}')
print(f'Testing Score: {lr.score(X_test, y_test)}')

Training Score: 1.0
Testing Score: 0.9860724233983287


In [42]:
##Model is overfit as training score is greater than testing set

In [43]:
#Multinomial Naive Bayes is appropriate when our features are variables that take on only positive integer counts


In [44]:
nb= MultinomialNB()
nb.fit(X_train,y_train)
predictions = nb.predict(X_test)


In [45]:
print(f'Training Score: {nb.score(X_train, y_train)}')
print(f'Testing Score: {nb.score(X_test, y_test)}')

Training Score: 1.0
Testing Score: 0.9888579387186629


In [46]:
print(confusion_matrix(y_test,predictions))
print('\n')
print(classification_report(y_test,predictions))

[[184   1]
 [  3 171]]


              precision    recall  f1-score   support

           0       0.98      0.99      0.99       185
           1       0.99      0.98      0.99       174

    accuracy                           0.99       359
   macro avg       0.99      0.99      0.99       359
weighted avg       0.99      0.99      0.99       359



In [47]:
##Accuracy score is 99%, which is better .

In [48]:
pipe_mnb =Pipeline([('bow',CountVectorizer()),
                    ('tfid',TfidfTransformer()),
               ('model',MultinomialNB())])

In [49]:
#data.head()

In [50]:
X_mnb= new_df['selftext']
y_mnb= new_df['subreddit']

In [51]:
X_train_mnb, X_test_mnb, y_train_mnb, y_test_mnb = train_test_split(X_mnb,
                                                    y_mnb,
                                                    test_size=0.33,
                                                    random_state=42)

In [52]:
pipe_mnb.fit(X_train_mnb,y_train_mnb)

Pipeline(memory=None,
         steps=[('bow',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfid',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('model',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [53]:
predictions = pipe_mnb.predict(X_test_mnb)

In [54]:
print(confusion_matrix(y_test_mnb,predictions))
print('\n')
print(classification_report(y_test_mnb,predictions))

[[196   1]
 [  1 161]]


              precision    recall  f1-score   support

           0       0.99      0.99      0.99       197
           1       0.99      0.99      0.99       162

    accuracy                           0.99       359
   macro avg       0.99      0.99      0.99       359
weighted avg       0.99      0.99      0.99       359



In [55]:
 
#My second Multinomial Naive Bayes model performed the best.  
#The accuracy score was 99.0% on training data and 98.0% on unclean data. 
#This means our model is slightly and probably inconsequentially overfit. 
#This also means that 99.0% of our posts will be accurately classified by our model.